
# **Install libraries**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install datasets tqdm pandas

In [3]:
!pip install sentencepiece

In [4]:
!pip install transformers

In [5]:
!pip install wandb

In [6]:
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm

In [7]:
# Check we have a GPU and check the memory size of the GPU
!nvidia-smi

Thu Sep  7 16:58:13 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# **Import packages**

In [8]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

from transformers import (
    AdamW,
    AutoModelForSeq2SeqLM,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# **Set a seed**

In [9]:
import random
import numpy as np
import torch
import datasets

In [10]:
def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)

set_seed(42)



```
# This is formatted as code
```

# ***C4-200M dataset***

In [11]:
pd.set_option('display.max_colwidth', None)

In [12]:
df = pd.read_csv('/content/drive/MyDrive/data/train_data.csv')
df.shape

(8032, 6)

In [13]:
df.head()

,Video Title,Genre,Comment,Error,Category,Correct Form
0,"ওবায়দুল কাদের বললেন, ‘খেলা হবে’; আর রুমিন ফারহানা বললেন, ‘আসেন খেলি’ ! | Quader | Rumeen Farhana",Politics,কাদের কি খেলব কাদের তো খেলতেই পারে না,1,Grammatical,কাদের কি খেলবে কাদের তো খেলতেই পারে না
1,পুলিশের গাড়ির ওপর চড়াও বিএনপির কর্মীরা | BNP | Channel 24,Politics,এসব করে আরো কোন ঠাসা হবে,1,Spelling,এসব করে আরো কোণঠাসা হবে
2,Ayub Bachchu | Ek Akash Tara | আইয়ুব বাচ্চু | এক আকাশ তারা | Official Music Video,Entertainment,যুগ যুগ ধরে আমাদের মনে গেথে থাকবে এ গান,0,NaN,যুগ যুগ ধরে আমাদের মনে গেথে থাকবে এ গান
3,যে প্রেম কাহিনী কোন বাধা মানেনি | BBC Bangla,Miscellaneous,অাচছা অাপু এলাজী থাকলে টিকা নেওয়া জাবেনা,1,Spelling,আচ্ছা আপু এলার্জী থাকলে টিকা নেওয়া যাবেনা
4,তুরস্কের চেয়ে ভয়াবহ ভূমিকম্পের ঝুঁকিতে বাংলাদেশ | BBC Bangla,News,হে আল্লাহ এই জালিমদের থেকে আমাদের সন্তান সন্তদের কে আপনি হেফাজত করেন,0,NaN,হে আল্লাহ এই জালিমদের থেকে আমাদের সন্তান সন্তদের কে আপনি হেফাজত করেন


In [14]:
from transformers import (
    AutoModelForSeq2SeqLM, AutoTokenizer,
    Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
  )

from torch.utils.data import Dataset, DataLoader

In [15]:
model_name = 'csebuetnlp/banglat5'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [16]:
def calc_token_len(example):
    return len(tokenizer(example).input_ids)

In [17]:
train_df = pd.read_csv('/content/drive/MyDrive/data/train_data.csv')
test_df = pd.read_csv('/content/drive/MyDrive/data/test_data.csv')

In [18]:
!pip install git+https://github.com/csebuetnlp/normalizer


  Cloning https://github.com/csebuetnlp/normalizer to /tmp/pip-req-build-jajouk2m
  Running command git clone --filter=blob:none --quiet https://github.com/csebuetnlp/normalizer /tmp/pip-req-build-jajouk2m
  Resolved https://github.com/csebuetnlp/normalizer to commit d405944dde5ceeacb7c2fd3245ae2a9dea5f35c9
  Preparing metadata (setup.py) ... done


In [19]:
from normalizer import normalize
train_df['Comment'] = train_df['Comment'].apply(normalize)
# train_df[['Comment','Correct Form']] = train_df[['Comment','Correct Form']].apply(normalize)
# test_df[['Comment','Correct Form']] = test_df[['Comment','Correct Form']].apply(normalize)

In [20]:
train_df['Correct Form'] = train_df['Correct Form'].apply(normalize)
test_df['Comment'] = test_df['Comment'].apply(normalize)
test_df['Correct Form'] = test_df['Correct Form'].apply(normalize)

In [21]:
train_df.shape, test_df.shape

((8032, 6), (2010, 6))

In [22]:
test_df['input_token_len'] = test_df['Comment'].apply(calc_token_len)

In [23]:
test_df.head()

,Video Title,Genre,Comment,Error,Category,Correct Form,input_token_len
0,নেত্রীর কথা শুনলে এখন হাজার হাজার শ্রমিক রাস্তায় নেমে যাবে' | Sheikh Sharhan Naser Tonmoy,Politics,আওয়ামী লীগের এতো লোক তাহলে কেন এত ভয় পায়,0,NaN,আওয়ামী লীগের এতো লোক তাহলে কেন এত ভয় পায়,10
1,RedMagic 8 Pro দেখে আমি তো অবাক 😮,Miscellaneous,প্রাইস টা বললে কি হতো রে ইমন,1,Code Switching,দাম টা বললে কি হতো রে ইমন,8
2,অস্থির বাঙালি Part 35😂 osthir bengali | funny video | funny facts | facts bangla,Entertainment,এত সুন্দর হাসি ভালো লাগল,1,Spelling,এত সুন্দর হাসি ভালো লাগলো,6
3,দেশে প্রথমবারের মতো চ্যানেল 24-এর পর্দায় সংবাদ পাঠ করলেন এআই 'অপরাজিতা' | AI Presenter | Channel 24,News,সময় চলে এসেছে আপনাদেরকে বিদায় জানাবার,0,NaN,সময় চলে এসেছে আপনাদেরকে বিদায় জানাবার,9
4,"মীনা, রাজু, মিঠুর মিমিক্রি করে তাক লাগিয়ে দিয়েছেন অথৈ | Oitijya Authoi Roy | Voice Artist | Somoy TV",Entertainment,চুল একটু বড় হলে ভালো হত না,1,Spelling,চুল একটু বড় হলে ভালো হতো না,8


In [24]:
test_df['input_token_len'].describe()

count    2010.000000
mean       11.428358
std         6.981458
min         4.000000
25%         7.000000
50%        10.000000
75%        13.000000
max       111.000000
Name: input_token_len, dtype: float64

### We will use a token length of 64 since it will cover the vast majority of examples

In [25]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [26]:
test_dataset

Dataset({
    features: ['Video Title', 'Genre', 'Comment', 'Error', 'Category', 'Correct Form', 'input_token_len'],
    num_rows: 2010
})

### Load the Dataset

In [27]:
from torch.utils.data import Dataset, DataLoader
class GrammarDataset(Dataset):
    def __init__(self, dataset, tokenizer,print_text=False):
        self.dataset = dataset
        self.pad_to_max_length = False
        self.tokenizer = tokenizer
        self.print_text = print_text
        self.max_len = 64

    def __len__(self):
        return len(self.dataset)


    def tokenize_data(self, example):
        input_, target_ = example['Comment'], example['Correct Form']

        # tokenize inputs
        tokenized_inputs = tokenizer(input_, pad_to_max_length=self.pad_to_max_length,
                                            max_length=self.max_len,
                                            return_attention_mask=True)

        tokenized_targets = tokenizer(target_, pad_to_max_length=self.pad_to_max_length,
                                            max_length=self.max_len,
                                            return_attention_mask=True)

        inputs={"input_ids": tokenized_inputs['input_ids'],
            "attention_mask": tokenized_inputs['attention_mask'],
            "labels": tokenized_targets['input_ids']
        }

        return inputs


    def __getitem__(self, index):
        inputs = self.tokenize_data(self.dataset[index])

        if self.print_text:
            for k in inputs.keys():
                print(k, len(inputs[k]))

        return inputs

In [28]:
dataset = GrammarDataset(test_dataset, tokenizer, True)
print(dataset[121])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


input_ids 4
attention_mask 4
labels 4
{'input_ids': [179, 1158, 3409, 1], 'attention_mask': [1, 1, 1, 1], 'labels': [179, 1158, 3409, 1]}


### Define Evaluator

In [29]:
!pip install rouge_score

In [30]:
from datasets import load_metric
rouge_metric = load_metric("rouge")

<ipython-input-30-76fa4d2ef6b0>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric("rouge")


### Train Model

In [31]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding='longest', return_tensors='pt')

In [32]:
!pip install transformers[torch]

In [33]:
!pip install accelerate -U

In [34]:
# defining training related arguments
batch_size = 16
args = Seq2SeqTrainingArguments(output_dir="/content/drive/MyDrive/weights",
                        evaluation_strategy="steps",
                        per_device_train_batch_size=batch_size,
                        per_device_eval_batch_size=batch_size,
                        learning_rate=2e-5,
                        num_train_epochs=8,
                        weight_decay=0.01,
                        save_total_limit=2,
                        predict_with_generate=True,
                        fp16 = True,
                        gradient_accumulation_steps = 6,
                        eval_steps = 500,
                        save_steps = 500,
                        load_best_model_at_end=True,
                        logging_dir="/logs",
                        report_to="wandb")

In [35]:
import nltk
nltk.download('punkt')
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [36]:
# defining trainer using 🤗
trainer = Seq2SeqTrainer(model=model,
                args=args,
                train_dataset= GrammarDataset(train_dataset, tokenizer),
                eval_dataset=GrammarDataset(test_dataset, tokenizer),
                tokenizer=tokenizer,
                data_collator=data_collator,
                compute_metrics=compute_metrics)

In [37]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
500,0.000000,nan,0.000000,0.000000,0.000000,0.000000,0.000000


TrainOutput(global_step=664, training_loss=0.0, metrics={'train_runtime': 665.7092, 'train_samples_per_second': 96.523, 'train_steps_per_second': 0.997, 'total_flos': 2344780364120064.0, 'train_loss': 0.0, 'epoch': 7.94})

In [38]:
trainer.save_model('bangla_gec_model')

In [39]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [44]:
!zip -r 'bangla_gec_model.zip' 'bangla_gec_model'

  adding: bangla_gec_model/ (stored 0%)
  adding: bangla_gec_model/generation_config.json (deflated 29%)
  adding: bangla_gec_model/training_args.bin (deflated 49%)
  adding: bangla_gec_model/spiece.model (deflated 60%)
  adding: bangla_gec_model/special_tokens_map.json (deflated 86%)
  adding: bangla_gec_model/config.json (deflated 48%)
  adding: bangla_gec_model/tokenizer.json (deflated 76%)
  adding: bangla_gec_model/pytorch_model.bin (deflated 53%)
  adding: bangla_gec_model/tokenizer_config.json (deflated 83%)


In [45]:
!mv bangla_gec_model.zip /content/drive/MyDrive/model

I have uploaded this model to HuggingFace Model Zoo and we can run inference using it

## Testing

In [51]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model_name = 'bangla_gec_model'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(torch_device)

def correct_grammar(input_text,num_return_sequences,input_len):
  batch = tokenizer([input_text],truncation=True,padding='max_length',max_length=input_len, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=input_len,num_beams=4, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

In [52]:
references,predictions = [],[]
test_d = test_df[test_df['Error']==1]
test_d
# for sentence in test_sentences:
#   print(f"input sentence:",sentence)
#   references.append(sentence)
#   output_sentence = correct_grammar(sentence,num_return_sequences=2,)
#   print("output sentence: ",output_sentence)
#   predictions.append(output_sentence[0])

,Video Title,Genre,Comment,Error,Category,Correct Form,input_token_len
1,RedMagic 8 Pro দেখে আমি তো অবাক 😮,Miscellaneous,প্রাইস টা বললে কি হতো রে ইমন,1,Code Switching,দাম টা বললে কি হতো রে ইমন,8
2,অস্থির বাঙালি Part 35😂 osthir bengali | funny video | funny facts | facts bangla,Entertainment,এত সুন্দর হাসি ভালো লাগল,1,Spelling,এত সুন্দর হাসি ভালো লাগলো,6
4,"মীনা, রাজু, মিঠুর মিমিক্রি করে তাক লাগিয়ে দিয়েছেন অথৈ | Oitijya Authoi Roy | Voice Artist | Somoy TV",Entertainment,চুল একটু বড় হলে ভালো হত না,1,Spelling,চুল একটু বড় হলে ভালো হতো না,8
5,ঘোড়া কীভাবে সাপের কামড় থেকে মানুষকে বাঁচায়? | Why horse is used for antivenom? | Jamuna TV,Miscellaneous,সমস্যা হলো সকল সরকারি হসপিটালে অন্টিভেনোম না পাওয়া,1,Spelling,সমস্যা হলো সকল সরকারি হসপিটালে এন্টিভ্যানম না পাওয়া,12
6,খেলা হবে' রাজনীতির মাঠে আবারও আলোচনায় কেন? | BBC Bangla,Politics,চেয়ার খেলা শুরু করে খেলার উদ্বোধন করলেন প্রধানমন্ত্রী লোকেরা মোজা পাইছি কাদের কাউয়া,1,Spelling,চেয়ার খেলা শুরু করে খেলার উদ্বোধন করলেন প্রধানমন্ত্রী লোকেরা মজা পেয়েছে কাদের কাউয়া,16
...,...,...,...,...,...,...,...
1996,"Liver Cirrhosis: লিভার সিরোসিস রোগের কারণ, লক্ষণ ও চিকিৎসা কী? | BBC Bangla",Miscellaneous,এই রুগটা কি বন্স গতো জানা বেন,1,Spelling,এই রোগটা কি বংশগতো জানাবেন,12
2003,সিরাহ ৭ – অ্যাবিসিনিয়া | Bangla Seerah,Miscellaneous,মাশাআল্লাহ এই লোকটি আমার প্রিয় মানুসের মধ্যে একজন,1,Spelling,মাশাআল্লাহ এই লোকটি আমার প্রিয় মানুষের মধ্যে একজন,12
2004,তুরস্কের চেয়ে ভয়াবহ ভূমিকম্পের ঝুঁকিতে বাংলাদেশ | BBC Bangla,News,আল্লাহ তুমি বনটিকে জান্নাতে তার সামী সাথে তাকতে দিয় আমিন,1,Multiple Errors,আল্লাহ তুমি বোনটিকে জান্নাতে তার স্বামীর সাথে থাকতে দিয়ো আমিন,15
2007,খালেদা জিয়া’র ব্যাপারে আর কত করবো ? প্রশ্ন প্রধানমন্ত্রীর | Sheikh Hasina | Khaleda Zia,Politics,আর কত করব তাও তো মরলনা,1,Spelling,আর কত করবো তাও তো মরলো না,9


In [53]:
test_d_sentence = test_d['Comment'].tolist()[:5]
test_d_len = test_d['input_token_len'].tolist()[:5]

In [54]:
type(test_d_len[0])

int

In [55]:
input_sentence = test_d_sentence[0]
print('Input sentence is : {}'.format(input_sentence))
correct_grammar(input_sentence,num_return_sequences=2,input_len=test_d_len[0])

Input sentence is : প্রাইস টা বললে কি হতো রে ইমন


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `1.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


['প্রাইস প্রাইস না পেলে প্রাইস প্রাইস', 'প্রাইস প্রাইস না বললে প্রাইস প্রাইস']

In [56]:
for i in range(len(test_d_sentence)):
  references.append(i)
  predictions.append(correct_grammar(test_d_sentence[i],num_return_sequences=2,input_len=test_d_len[i])[0])

In [58]:
results = rouge_metric.compute(predictions=predictions, references=references)

In [57]:
print(list(results.keys()))

['rouge1', 'rouge2', 'rougeL', 'rougeLsum']


In [48]:
print(results["rouge1"])

AggregateScore(low=Score(precision=0.0, recall=0.0, fmeasure=0.0), mid=Score(precision=0.0, recall=0.0, fmeasure=0.0), high=Score(precision=0.0, recall=0.0, fmeasure=0.0))


In [59]:
print(results["rouge2"])

AggregateScore(low=Score(precision=0.0, recall=0.0, fmeasure=0.0), mid=Score(precision=0.0, recall=0.0, fmeasure=0.0), high=Score(precision=0.0, recall=0.0, fmeasure=0.0))


In [60]:
print(results["rougeL"])

AggregateScore(low=Score(precision=0.0, recall=0.0, fmeasure=0.0), mid=Score(precision=0.0, recall=0.0, fmeasure=0.0), high=Score(precision=0.0, recall=0.0, fmeasure=0.0))
